<a href="https://colab.research.google.com/github/yinlongTh/Python/blob/main/GW_DistanceNTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cost to Serve Customer**

**Task : ETL**
Work Pack Job listing showing the times the jobs were schedules and completed
Filtered from 01 Oct 2023 until 31 Oct 2023 11:59

In [ ]:
!pip install googlemaps
import googlemaps
import pandas as pd

gmaps = googlemaps.Client(key='')

In [ ]:
class Vehicle :

    V1CostPerKm = 3 #GBP/Km
    V2CostPerKm = 4 #GBP/Km

    def __init__(self, CarID, VehicleType):
        self.CarID = CarID
        self.VehicleType = VehicleType

    def FuelCost(self, Distance): #Calculate fuel cost for a distance
        if self.VehicleType == "V1":
            fuelcost = self.V1CostPerKm*Distance
        else:
            fuelcost = self.V2CostPerKm*Distance

        return fuelcost

In [ ]:
class Customer:

  def __init__(self, Name, Location, NumberOfCollection):
    self.Name = Name
    self.Location = Location
    self.NumberOfCollection = NumberOfCollection #Number of Scheduled Collection per month
    self.OpTime = 60 #minutes of operational time
    self.NumberOfStaff = 2

  def NearestEmptyPoint(self,EmptyPoint): #Find nearest empty point and distance
    DummyDistance = []
    DummyTime = []
    for point in EmptyPoint:
      DistanceNDuration = gmaps.distance_matrix([self.Location], [point], mode='driving')['rows'][0]['elements'][0] #Distance and Duration
      if len(DistanceNDuration) == 1: #No value return error
        DummyDistance.append(9999)
        DummyTime.append(9999)
      else:
        DummyDistance.append(DistanceNDuration['distance']['value']/1000) #Kilometer
        DummyTime.append(DistanceNDuration['duration']['value']/60) #Minutes

      DistanceNDuration.clear()

    return min(DummyDistance), min(DummyTime)

  def YearFuelNTime(self,EmptyPoint,vehicle):

   distance, traveltime = self.NearestEmptyPoint(EmptyPoint)

   yearfuelcost = vehicle.FuelCost((self.NumberOfCollection*12)*(distance*2)) #a year speand on fuel
   yearservicetime = (self.NumberOfCollection*12)*(self.OpTime+(traveltime*2))

   return yearfuelcost, yearservicetime






**Test Code Sample**

In [ ]:
#Empty point coordinations
EmptyPoint = ["49.4261402973603 -2.6340230767835",
              "49.4261421009836 -2.63413501494324",
              "49.4312981077463 -2.59882868001964",
              "49.4313926762813 -2.59877334865725",
              "49.4631838692045 -2.58674832520885",
              "49.4641906933696 -2.58628128450619",
              "49.4632300263327 -2.57465445377643",
              "49.4632193395457 -2.57461209667111",
              "49.4441753734301 -2.56337612392802",
              "49.4442684873835 -2.56341905291427",
              "49.4443626785274 -2.5634633642399",
              "49.4773440324164 -2.53349462586062",
              "49.4773245519672 -2.53346424963821",
              "49.4773049451306 -2.53343436989802",
              "49.4828183412968 -2.51064777314559",
              "49.4827676680183 -2.51082266502041",
              "49.4827370112463 -2.51092873111758",
              "49.4827026574825 -2.51104762431695"]

**Real Data Import**

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_id = '1KTUZpGRhMDDrkFMas3xT7mJ43W16K1B1'
downloaded = drive.CreateFile({'id': file_id})

In [ ]:
# Download the file to a local disk as 'YTPGCollectionTimes.xlsx'.
downloaded.GetContentFile('YTPGCollectionTimes.xlsx')

In [ ]:
# Check imported file
!ls -lha YTPGCollectionTimes.xlsx

-rw-r--r-- 1 root root 310K Nov 30 14:52 YTPGCollectionTimes.xlsx


In [ ]:
!pip install -q xlrd

import pandas as pd
df = pd.read_excel('YTPGCollectionTimes.xlsx', sheet_name="Analysed")
df['Distance Nearest Empty Point'] = ""
df['Total Travel Time'] = ""
df

,Number of Crew,Job ID,UPRN,Location,Number of Collection,Distance Nearest Empty Point,Total Travel Time
0,2,611412,1000937,49.45816 -2.637484,9,,
1,2,611349,1001013,49.458612 -2.603388,12,,
2,2,610316,1001420,49.431568 -2.592866,4,,
3,1,629581,1002249,49.483658 -2.56137,16,,
4,2,610845,1002507,49.485761 -2.562463,4,,
...,...,...,...,...,...,...,...
1744,2,630798,22410,49.449425 -2.61109,2,,
1745,2,618705,23076,49.456371 -2.628464,2,,
1746,2,634366,23495,49.44117 -2.617253,1,,
1747,2,634287,23688,49.458789 -2.627234,1,,


In [ ]:
#Data Checking
newCus = Customer(df.loc[0]['UPRN'],df.loc[0]['Location'],df.loc[0]['Number of Collection'])
dmDistance,dmTime = newCus.NearestEmptyPoint(EmptyPoint)
print(dmDistance,dmTime)

4.537 8.45


In [ ]:
#Compute nearest distance and travel time
for index, row in df.iterrows():
  DummyCustomer = Customer(row['UPRN'],row['Location'],row['Number of Collection'])
  distance, time = DummyCustomer.NearestEmptyPoint(EmptyPoint)

  df.at[index, 'Distance Nearest Empty Point'] = distance
  df.at[index, 'Total Travel Time'] = time


In [ ]:
#Data Checking
df

,Number of Crew,Job ID,UPRN,Location,Number of Collection,Distance Nearest Empty Point,Total Travel Time
0,2,611412,1000937,49.45816 -2.637484,9,4.61,8.566667
1,2,611349,1001013,49.458612 -2.603388,12,1.841,3.95
2,2,610316,1001420,49.431568 -2.592866,4,0.572,1.2
3,1,629581,1002249,49.483658 -2.56137,16,3.351,7.35
4,2,610845,1002507,49.485761 -2.562463,4,3.614,7.266667
...,...,...,...,...,...,...,...
1744,2,630798,22410,49.449425 -2.61109,2,3.054,5.483333
1745,2,618705,23076,49.456371 -2.628464,2,3.767,6.383333
1746,2,634366,23495,49.44117 -2.617253,1,2.588,5.05
1747,2,634287,23688,49.458789 -2.627234,1,3.843,6.85


In [ ]:
#Export to excel
from google.colab import drive
drive.mount('drive')
df.to_csv('EmyptyPointNTravelTime.csv')
!cp EmyptyPointNTravelTime.csv "drive/My Drive/"

Mounted at drive


End of task

---

**Task : ETL**
Data Source : Schedules Listing
Filtered from 01 Nov 2022 until 31 Oct 2023 11:59

In [ ]:
file_id = '1yrycWgxEaS2X5CpTbqDfHZctDAQi4WGR'
downloaded = drive.CreateFile({'id': file_id})

In [ ]:
downloaded.GetContentFile('YLPGGWBAData.xlsx')

In [ ]:
!pip install -q xlrd

import pandas as pd
df = pd.read_excel('YLPGGWBAData.xlsx', sheet_name="Analyzed")
df['Distance Nearest Empty Point'] = ""
df['Total Travel Time'] = ""
df

,Number of Crew,UPRN,Frequency per Year,Location,Distance Nearest Empty Point,Total Travel Time
0,2,1000040,13,GY2 4WG,,
1,2,1000054,26,GY6 8AA,,
2,2,1000056,52,GY6 8UN,,
3,2,1000132,52,GY7 9ND,,
4,2,1000163,312,GY3 5BY,,
...,...,...,...,...,...,...
5117,1,9921005990,26,GY7 9JY,,
5118,2,9930008994,13,GY2 4FQ,,
5119,1,9930021328,52,GY6 8SH,,
5120,2,9930023586,52,GY7 9UQ,,


In [ ]:
for index, row in df.iterrows():
 # if str(row['Total Travel Time']) != '': #Skip already calculated
 #   print("-")
 # else:
    DummyCustomer = Customer(row['UPRN'],row['Location'],row['Frequency per Year']/12)

    distance, time = DummyCustomer.NearestEmptyPoint(EmptyPoint)

    df.at[index, 'Distance Nearest Empty Point'] = distance
    df.at[index, 'Total Travel Time'] = time

In [ ]:
#Export to excel
from google.colab import drive
drive.mount('drive', force_remount=True)
df.to_csv('TestData.csv')
!cp TestData.csv "drive/My Drive/"

Mounted at drive


End of task

---